<a href="https://colab.research.google.com/github/ldtownsend/DS7-Portfolio-Project-Energy/blob/master/First_Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [84]:
import os, sys
in_colab = 'google.colab' in sys.modules

# If you're in Colab...
if in_colab:
    # Pull files from Github repo
    os.chdir('/content')
    !git init .
    !git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Applied-Modeling.git
    !git pull origin master
    
    # Install required python packages
    !pip install -r requirements.txt
    
    # Change into directory for module
    os.chdir('module3')

Reinitialized existing Git repository in /content/.git/
fatal: remote origin already exists.
From https://github.com/LambdaSchool/DS-Unit-2-Applied-Modeling
 * branch            master     -> FETCH_HEAD
Already up to date.


##Exploring Time-Series data

In [147]:
import pandas as pd

url = 'https://raw.githubusercontent.com/ldtownsend/DS7-Portfolio-Project-Energy/master/df_master_monthly.csv'
df = pd.read_csv(url)
df.head()

,Year,"Total Energy Consumed by the Commercial Sector, Monthly","Total Energy Consumed by the Industrial Sector, Monthly","Total Energy Consumed by the Residential Sector, Monthly","Total Energy Consumed by the Transportation Sector, Monthly","Average Retail Price of Electricity, Commercial, Monthly","Average Retail Price of Electricity, Industrial, Monthly","Average Retail Price of Electricity, Residential, Monthly","Average Retail Price of Electricity, Transportation, Monthly","Average Retail Price of Electricity, Total, Monthly"
0,201905,1409.636,2732.911,1374.866,2416.107,10.53,6.71,13.32,9.51,10.43
1,201904,1358.030,2614.072,1370.355,2315.086,10.51,6.53,13.26,9.58,10.31
2,201903,1604.748,2740.601,1980.916,2357.658,10.44,6.73,12.83,9.33,10.38
3,201902,1592.656,2536.770,2125.867,2103.190,10.52,6.68,12.70,10.01,10.43
4,201901,1819.267,2893.769,2509.892,2285.389,10.29,6.58,12.47,9.80,10.29


In [148]:
df_commercial = df[['Year', 'Total Energy Consumed by the Commercial Sector, Monthly']]
df_commercial.rename(columns={'Year': 'Period', 'Total Energy Consumed by the Commercial Sector, Monthly' : 'Commercial Sector'})
df_commercial.head()

,Year,"Total Energy Consumed by the Commercial Sector, Monthly"
0,201905,1409.636
1,201904,1358.030
2,201903,1604.748
3,201902,1592.656
4,201901,1819.267


In [149]:
df_commercial = df_commercial.rename(columns={'Year': 'Period', 'Total Energy Consumed by the Commercial Sector, Monthly' : 'Commercial Sector'})
df_commercial.head()

,Period,Commercial Sector
0,201905,1409.636
1,201904,1358.030
2,201903,1604.748
3,201902,1592.656
4,201901,1819.267


Create some time-series features:

In [0]:
df_commercial['Period'] = df_commercial['Period'].astype(str)

In [151]:
df_commercial['Year'] = df_commercial['Period'].str[:4].astype(int)
df_commercial['Month'] = df_commercial['Period'].str[4:]
df_commercial['Period'] = df_commercial['Period'].astype(int)

df_commercial.head()

,Period,Commercial Sector,Year,Month
0,201905,1409.636,2019,05
1,201904,1358.030,2019,04
2,201903,1604.748,2019,03
3,201902,1592.656,2019,02
4,201901,1819.267,2019,01


In [152]:
df_commercial['Period'] = df_commercial['Period'].astype(int)
df_commercial.dtypes

Period                 int64
Commercial Sector    float64
Year                   int64
Month                 object
dtype: object

In [0]:
train = df_commercial[df_commercial['Period'] <= 201612]
val = df_commercial[(df_commercial['Period'] <= 201712) & (df_commercial['Period'] >= 201701)]
val_month = df_commercial[df_commercial['Period'] == 201701]
test = df_commercial[df_commercial['Period'] >= 201801]

In [154]:
test.shape

(17, 4)

In [0]:
# Define the target and features, ultra simple for this version.
target = 'Commercial Sector'
features = ['Year', 'Month']

# Arrange data into X features and y target vector
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]

X_val_month = val_month[features]
y_val_month = val_month[target]

X_test = test[features]

In [0]:
import category_encoders as ce
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.pipeline import make_pipeline

In [157]:
# Start with a linear regression
lr = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True), 
    SimpleImputer(strategy='median'), 
    LinearRegression()
)

lr.fit(X_train, y_train)
lr_pred = lr.predict(X_val)

print(r2_score(y_val, lr_pred), mean_absolute_error(y_val, lr_pred), mean_squared_error(y_val, lr_pred))
# print('Linear Regression R^2', lr.score(X_val, y_val)) # This doesn't seem right at all...

-0.9438684726329838 163.85806263213584 30509.73829751281


In [158]:
lr.predict(X_val)

array([1812.99929559, 1614.31268195, 1554.82554559, 1540.10306831,
       1649.4427274 , 1654.87445468, 1586.25729559, 1550.42277286,
       1553.46250013, 1707.60168195, 1748.5047274 , 1890.29900013])

In [162]:
y_val.values

array([1756.841, 1494.822, 1413.532, 1376.382, 1484.234, 1527.884,
       1435.648, 1386.268, 1322.234, 1559.824, 1432.607, 1706.533])

In [159]:
pipeline = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True), 
#     SimpleImputer(strategy='median'), 
    RandomForestRegressor(n_estimators=100, random_state=15, n_jobs=-1)
)

# Fit on train, score on val
pipeline.fit(X_train, y_train)
print('Random Forest R^2', pipeline.score(X_val, y_val))

Random Forest R^2 0.7530962084486873


In [160]:
rf_pred = pipeline.predict(X_val)

print(r2_score(y_val, rf_pred), mean_absolute_error(y_val, rf_pred), mean_squared_error(y_val, rf_pred))

0.7530962084486869 49.09063583333303 3875.246793159187


In [163]:
rf_pred

array([1681.98312, 1441.66134, 1398.49305, 1411.04321, 1539.18383,
       1544.18051, 1458.39468, 1383.48983, 1346.6929 , 1478.59865,
       1580.75142, 1767.30207])

In [120]:
print('X_train shape before encoding', X_train.shape)

encoder = pipeline.named_steps['onehotencoder']
encoded = encoder.transform(X_train)
print('X_train shape after encoding', encoded.shape)

X_train shape before encoding (533, 2)
X_train shape after encoding (533, 13)


In [138]:
encoded.head()

,Year,Month_05,Month_04,Month_03,Month_02,Month_01,Month_12,Month_11,Month_10,Month_09,Month_08,Month_07,Month_06
24,2017,1,0,0,0,0,0,0,0,0,0,0,0
25,2017,0,1,0,0,0,0,0,0,0,0,0,0
26,2017,0,0,1,0,0,0,0,0,0,0,0,0
27,2017,0,0,0,1,0,0,0,0,0,0,0,0
28,2017,0,0,0,0,1,0,0,0,0,0,0,0
